In [1]:
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf  #using tf v.1.14.0
import keras
import numpy as np

Using TensorFlow backend.


In [3]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [4]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=88000)

In [5]:
np.load = np_load_old

In [6]:
_word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in _word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

In [7]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [8]:
model = keras.Sequential()
model.add(keras.layers.Embedding(88000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
x_train = train_data[10000:]
x_val = train_data[:10000]

y_train = train_labels[10000:]
y_val = train_labels[:10000]

In [12]:
fitModel = model.fit(x_train, y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)


Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 3s 167us/step - loss: 0.6916 - acc: 0.6041 - val_loss: 0.6895 - val_acc: 0.5659
Epoch 2/40
15000/15000 [==============================] - 2s 102us/step - loss: 0.6848 - acc: 0.7298 - val_loss: 0.6801 - val_acc: 0.7558
Epoch 3/40
15000/15000 [==============================] - 2s 103us/step - loss: 0.6703 - acc: 0.7707 - val_loss: 0.6623 - val_acc: 0.7447
Epoch 4/40
15000/15000 [==============================] - 2s 103us/step - loss: 0.6442 - acc: 0.7849 - val_loss: 0.6334 - val_acc: 0.7839
Epoch 5/40
15000/15000 [==============================] - 2s 106us/step - loss: 0.6034 - acc: 0.8133 - val_loss: 0.5917 - val_acc: 0.7976
Epoch 6/40
15000/15000 [==============================] - 2s 103us/step - loss: 0.5517 - acc: 0.8308 - val_loss: 0.5459 - val_acc: 0.8095
Epoch 7/40
15000/15000 [==============================] - 2s 105us/step - loss: 0.4954 - acc: 0.8509 - val_loss: 0.4975 -

In [13]:
results = model.evaluate(test_data, test_labels)
print(results)

25000/25000 [==============================] - 1s 22us/step
[0.3405089078140259, 0.87144]


In [14]:
model.save("model.h5")

In [15]:
model = keras.models.load_model("model.h5")

In [16]:
def review_encode(s):
    encoded = [1]

    for word in s:
        if word.lower() in word_index:
            encoded.append(word_index[word.lower()])
        else:
            encoded.append(2)

    return encoded

In [17]:
with open("test.txt", encoding="utf-8") as f:
    for line in f.readlines():
        nline = line.replace(",", "").replace(".", "").replace("(", "").replace(")", "").replace(":", "").replace("\"","").strip().split(" ")
        encode = review_encode(nline)
        encode = keras.preprocessing.sequence.pad_sequences([encode], value=word_index["<PAD>"], padding="post", maxlen=250)
        predict = model.predict(encode)
        print(line)
        print(encode)
        print(predict[0])

Of all the animation classics from the Walt Disney Company, there is perhaps none that is more celebrated than "The Lion King." Its acclaim is understandable: this is quite simply a glorious work of art."The Lion King" gets off to a fantastic start. The film's opening number, "The Circle of Life," is outstanding. The song lasts for about four minutes, but from the first sound, the audience is floored. Not even National Geographic can capture something this beautiful and dramatic. Not only is this easily the greatest moment in film animation, this is one of the greatest sequences in film history. The story that follows is not as majestic, but the film has to tell a story. Actually, the rest of the film holds up quite well. The story takes place in Africa, where the lions rule. Their king, Mufasa (James Earl Jones) has just been blessed with a son, Simba (Jonathan Taylor Thomas), who goes in front of his uncle Scar (Jeremy Irons) as next in line for the throne. Scar is furious, and sets 